In [1]:
# This notebook illustrates the halfar ice model

# Import funman related code
import os
from pathlib import Path
from funman import FunmanResults
import json
from funman import Point, Box, Parameter
from typing import List, Dict
from funman.api.run import Runner

RESOURCES = os.path.join(
   os.getcwd(), "../../resources"
)
EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "halfar")
MODEL_PATH = os.path.join(EXAMPLE_DIR, "halfar.json")
REQUEST_PATH = os.path.join(EXAMPLE_DIR, "halfar_request.json")


def summarize_results(variables, results):
   points = results.points()
   boxes = results.parameter_space.boxes()

   print(
      f"{len(points)} Points (+:{len(results.parameter_space.true_points())}, -:{len(results.parameter_space.false_points())}), {len(boxes)} Boxes (+:{len(results.parameter_space.true_boxes)}, -:{len(results.parameter_space.false_boxes)})"
   )
   if points and len(points) > 0:
      point: Point = points[-1]
      parameters: Dict[Parameter, float] = results.point_parameters(point)
      results.plot(variables=variables, label_marker={"true":",", "false": ","},  xlabel="Time", ylabel="Height", legend=variables,label_color={"true": "g", "false":"r"})
      print(f"A = {results.parameter_space.points()[0].values['A']:.5f}")
      print(parameters)
      print(results.dataframe([point]))
   else:
      # if there are no points, then we have a box that we found without needing points
      box = boxes[0]
      print(json.dumps(box.explain(), indent=4))


# %load_ext autoreload
# %autoreload 2

In [2]:
# Use a five point model with no constraints

num_disc = 5

MODEL_PATH = os.path.join("../../resources/amr/halfar/hand_generated", f"halfar_{num_disc}.json")

locations = list(range(num_disc))
height_bounds = [
            {"name": f"pos_h_{i}",
         "variable": f"h_{i}",
         "interval": {"lb": 0, "ub": 1.01}
         }
    for i in locations
]

request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0,10, 1)}
            ],
        },
        
    ],
    "parameters":[
        {"name": "A",
         "label":"all",
         "interval": {"lb":1e-20, "ub":1e-5}}
        # "interval": {"lb":0, "ub":1}}
    ],
    "constraints": height_bounds + 
    [
        # {"name": "LHS_slope",
        #     "variables": ["h_1", "h_0"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }, 
        # {"name": "RHS_slope",
        #     "variables": ["h_3", "h_4"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }


    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-3,
        "verbosity": 20,
        # "dreal_mcts": False,
        "dreal_precision": 0.1,
        "save_smtlib": "./out",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}
variables = [f"h_{d}" for d in range(num_disc)]

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
    dump_plot=True,
    parameters_to_plot=["A", "timestep"],
    point_plot_config={"variables":variables, "label_marker":{"true":",", "false": ","},  "xlabel":"Time", "ylabel":"Height", "legend":variables,"label_color":{"true": "g", "false":"r"}},
    num_points=None,
    
)

summarize_results(variables, results)




Saving smtlib file: ./out/box_search_0_128.smt2
Saving smtlib file: ./out/box_search_0_129.smt2
Saving smtlib file: ./out/box_search_0_130.smt2
Saving smtlib file: ./out/box_search_0_131.smt2
Saving smtlib file: ./out/box_search_0_132.smt2


2023-12-01 16:28:36,642 - funman.api.run - INFO - Dumping results to ./out/d06c4301-7759-40f1-9a11-f39f88e6fdef.json
2023-12-01 16:28:36,651 - funman.search.box_search - ERROR - Traceback (most recent call last):
  File "/home/danbryce/funman/src/funman/search/box_search.py", line 1118, in _expand
    ) = self._get_true_points(
  File "/home/danbryce/funman/src/funman/search/box_search.py", line 955, in _get_true_points
    points, explanation = self._get_points(
  File "/home/danbryce/funman/src/funman/search/box_search.py", line 786, in _get_points
    result = self.invoke_solver(solver)
  File "/home/danbryce/funman/src/funman/search/search.py", line 122, in invoke_solver
    result = s.solve()
  File "/home/danbryce/funman_venv/lib/python3.8/site-packages/pysmt/decorators.py", line 64, in clear_pending_pop_wrap
    return f(self, *args, **kwargs)
  File "/home/danbryce/funman/auxiliary_packages/funman_dreal/src/funman_dreal/solver.py", line 626, in solve
    raise e
  File "/home/d

0 Points (+:0, -:0), 0 Boxes (+:0, -:0)


IndexError: list index out of range

In [ ]:
results.dataframe(points=results.parameter_space.points()[4:])

In [ ]:
# Use a five point model with no constraints

num_disc = 10

MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")

height_bounds = [
            {"name": f"pos_h_{i}",
         "variable": f"h_{i}",
         "interval": {"lb": 0, "ub": 1.01}
         },
    for i in range(num_disc)
]

request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 10, 1)}
            ],
        },
        
    ],
    "parameters":[
        {"name": "A",
         "label":"all",
        #  "interval": {"lb":1e-18, "ub":1e-14}}
        "interval": {"lb":0, "ub":1}}
    ],
    "constraints": 
    height_bounds + [
   
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        {"name": "RHS_slope",
            "variables": ["h_8", "h_9"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }


    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-5,
        "verbosity": 10,
        "dreal_mcts": True,
        # "dreal_precision": 1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}
variables = [f"h_{d}" for d in range(num_disc)]

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
    dump_plot=True,
    parameters_to_plot=["A", "timestep"],
    point_plot_config={"variables":variables, "label_marker":{"true":",", "false": ","},  "xlabel":"Time", "ylabel":"Height", "legend":variables,"label_color":{"true": "g", "false":"r"}},
    num_points=None
)

summarize_results(variables, results)




In [ ]:
# Use a five point model with no constraints

num_disc = 5
MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 20, 1)}
            ],
        },
        
    ],
    "parameters":[
        {"name": "gamma",
         "label":"all",
         "interval": {"lb":0, "ub":0.5}}
    ],
    "constraints": [
        {"name": "pos_h_0",
         "variable": "h_0",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_1",
         "variable": "h_1",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_2",
         "variable": "h_2",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_3",
         "variable": "h_3",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_4",
         "variable": "h_4",
         "interval": {"lb": 0}
        },
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        {"name": "RHS_slope",
            "variables": ["h_3", "h_4"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }


    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-5,
        "verbosity": 30,
        "dreal_mcts": True,
        "dreal_precision": 1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}
variables = [f"h_{d}" for d in range(num_disc)]
   
# points = results.points()
# boxes = results.parameter_space.boxes()

# print(
#     f"{len(points)} Points (+:{len(results.parameter_space.true_points())}, -:{len(results.parameter_space.false_points())}), {len(boxes)} Boxes (+:{len(results.parameter_space.true_boxes)}, -:{len(results.parameter_space.false_boxes)})"
# )

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
    dump_plot=True,
    parameters_to_plot=["gamma", "timestep"],
    point_plot_config={"variables":variables, "label_marker":{"true":",", "false": ","},  "xlabel":"Time", "ylabel":"Height", "legend":variables,"label_color":{"true": "g", "false":"r"}},
    num_points=1
)
# summarize_results(num_disc, results)




In [ ]:
# Use a ten point model with no constraints

num_disc = 10
MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 25, 1)}
            ],
        },
        
    ],
    "parameters":[
        {"name": "gamma",
         "label":"all",
         "interval": {"lb":0, "ub":0.5}}
    ],
    "constraints": [        
        {"name": "pos_h_0",
         "variable": "h_0",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_1",
         "variable": "h_1",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_2",
         "variable": "h_2",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_3",
         "variable": "h_3",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_4",
         "variable": "h_4",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_5",
         "variable": "h_5",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_6",
         "variable": "h_6",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_7",
         "variable": "h_7",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_8",
         "variable": "h_8",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_9",
         "variable": "h_9",
         "interval": {"lb": 0}
        },
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        {"name": "RHS_slope",
            "variables": ["h_8", "h_9"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }

        ],
    "config": {
               "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-5,
        "verbosity": 30,
        "dreal_mcts": True,
        "dreal_precision": 1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}
variables = [f"h_{d}" for d in range(num_disc)]
# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
    dump_plot=True,
    parameters_to_plot=["gamma", "timestep"],
    point_plot_config={"variables":variables, "label_marker":{"true":",", "false": ","},  "xlabel":"Time", "ylabel":"Height", "legend":variables,"label_color":{"true": "g", "false":"r"}},
    num_points=1
)

In [ ]:
# Use a ten point model with no constraints

num_disc = 20
MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 25, 1)}
            ],
        },
        
    ],
    "parameters":[
        {"name": "gamma",
         "label":"all",
         "interval": {"lb":0, "ub":0.5}}
    ],
    "constraints": [        
        {"name": "pos_h_0",
         "variable": "h_0",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_1",
         "variable": "h_1",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_2",
         "variable": "h_2",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_3",
         "variable": "h_3",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_4",
         "variable": "h_4",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_5",
         "variable": "h_5",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_6",
         "variable": "h_6",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_7",
         "variable": "h_7",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_8",
         "variable": "h_8",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_9",
         "variable": "h_9",
         "interval": {"lb": 0}
        },
                {"name": "pos_h_10",
         "variable": "h_10",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_11",
         "variable": "h_11",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_12",
         "variable": "h_12",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_13",
         "variable": "h_13",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_14",
         "variable": "h_14",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_15",
         "variable": "h_15",
         "interval": {"lb": 0}
         },
        {"name": "pos_h_16",
         "variable": "h_16",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_17",
         "variable": "h_17",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_18",
         "variable": "h_18",
         "interval": {"lb": 0}
        },
        {"name": "pos_h_19",
         "variable": "h_19",
         "interval": {"lb": 0}
        },
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        {"name": "RHS_slope",
            "variables": ["h_18", "h_19"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }

        ],
    "config": {
               "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-5,
        "verbosity": 30,
        "dreal_mcts": True,
        "dreal_precision": 1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}
variables = [f"h_{d}" for d in range(num_disc)]
# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
    dump_plot=True,
    parameters_to_plot=["gamma", "timestep"],
    point_plot_config={"variables":variables, "label_marker":{"true":",", "false": ","},  "xlabel":"Time", "ylabel":"Height", "legend":variables,"label_color":{"true": "g", "false":"r"}},
    num_points=1
)

In [ ]:
# Use a five point model with no constraints

num_disc = 10
# MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 5, 1)}
            ],
        },
        
    ],
    "constraints": [
        {"name": "non-negative_h_0",
         "variable": "h_0",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
         },
        {"name": "non-negative_h_1",
         "variable": "h_1",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_2",
         "variable": "h_2",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_3",
         "variable": "h_3",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_4",
         "variable": "h_4",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        # {"name": "LHS_slope",
        #     "variables": ["h_1", "h_0"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }, 
        # {"name": "RHS_slope",
        #     "variables": ["h_3", "h_4"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }


    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-1,
        "verbosity": 10,
        "dreal_mcts": True,
        "dreal_precision": 0.1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
)
summarize_results(num_disc, results)




In [ ]:
# Use a five point model with no constraints

num_disc = 5
# MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 8, 1)}
            ],
        },
        
    ],
    "constraints": [
        {"name": "non-negative_h_0",
         "variable": "h_0",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
         },
        {"name": "non-negative_h_1",
         "variable": "h_1",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_2",
         "variable": "h_2",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_3",
         "variable": "h_3",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non-negative_h_4",
         "variable": "h_4",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "LHS_slope (h_0 <= h_1)",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        {"name": "RHS_slope (h_3 >= h_4)",
            "variables": ["h_3", "h_4"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }


    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-1,
        "verbosity": 10,
        "dreal_mcts": True,
        "dreal_precision": 0.1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
)
summarize_results(num_disc, results)




In [ ]:
# Use a five point model with no constraints

num_disc = 5
# MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 8, 1)}
            ],
        },
        
    ],
    "constraints": [
        {"name": "non_negative_h_0",
         "variable": "h_0",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
         },
        {"name": "non_negative_h_1",
         "variable": "h_1",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_2",
         "variable": "h_2",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_3",
         "variable": "h_3",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_4",
         "variable": "h_4",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },

        # (h_0 <= h_1)
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        # (h_3 >= h_4)
        {"name": "RHS_slope",
            "variables": ["h_3", "h_4"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        },


     {"name": "melt_h_2",
         "variable": "h_2",
         "interval": {"lb": 0, "ub": .8},
         "timepoints": {"lb": 5}
        },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-1,
        "verbosity": 10,
        "dreal_mcts": True,
        "dreal_precision": 0.1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
)
summarize_results(num_disc, results)




In [ ]:
# Use a five point model with no constraints

num_disc = 5
# MODEL_PATH = os.path.join("../..", f"halfar_{num_disc}.json")


request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0, 8, 1)}
            ],
        },
        
    ],
    "constraints": [
        {"name": "non_negative_h_0",
         "variable": "h_0",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
         },
        {"name": "non_negative_h_1",
         "variable": "h_1",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_2",
         "variable": "h_2",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_3",
         "variable": "h_3",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },
        {"name": "non_negative_h_4",
         "variable": "h_4",
         "interval": {"lb": 0},
         "timepoints":{"lb":0}
        },

        # (h_0 <= h_1)
        {"name": "LHS_slope",
            "variables": ["h_1", "h_0"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        }, 
        # (h_3 >= h_4)
        {"name": "RHS_slope",
            "variables": ["h_3", "h_4"],
            "weights": [1, -1],
            "additive_bounds": {"lb": 0},
            "timepoints": {"lb": 0}
        },


     {"name": "melt_h_2",
         "variable": "h_2",
         "interval": {"lb": 0, "ub": 1}
        },

        ],
    "config": {
        "use_compartmental_constraints": False,
        "normalization_constant": 1.0,
        "tolerance": 1e-1,
        "verbosity": 10,
        "dreal_mcts": True,
        "dreal_precision": 0.1,
        # "save_smtlib": "halfar.smt2",
        "substitute_subformulas": False,
        "series_approximation_threshold": None,
        "dreal_log_level": "none",
        "profile": False,
    },
}

# Use request_dict
results = Runner().run(
    MODEL_PATH,
    request_dict,
    # REQUEST_PATH,
    description="Halfar demo",
    case_out_dir="./out",
)
summarize_results(num_disc, results)


